In [1]:
from graph2net.trainers import gen_and_validate,generate,model_validate,full_model_run
from graph2net.data_loaders import *
from graph2net.graph_generators import *
from graph2net.archetypes import inception
from graph2net.helpers import *
from graph2net.notifier import notify_me
import logging
import numpy as np
import pandas as pd
import pickle as pkl
import time
import os
import random
import matplotlib.pyplot as plt


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(filename='logs/model_testbed.log', level=logging.INFO)

%load_ext autoreload
%autoreload 2

In [2]:
data = load_data(batch_size=256)

In [3]:
lr_schedule512 = {
    'type': 'cosine',
    'lr_min': .00001,
    'lr_max': .01,
    't_0': 1,
    't_mult': 2
}

In [6]:
def micro_net(cell):
    print("== micro ==")
    t_start = time.time()
    model = gen_and_validate([cell],data,scale=2,cell_types=[1,1])
    loss, correct, preds,acc_preds  = full_model_run(model,
                                          data=data,
                                          epochs=8,
                                          lr=.01,
                                          momentum=.9,
                                          weight_decay=1e-4,
                                          lr_schedule=lr_schedule512,
                                          drop_path=False,
                                          log=True,
                                          track_progress=True,
                                          prefix="Micro",
                                          verbose=False)
    return correct,time.time()-t_start

def macro_net(cell):
    print("\n== macro ==")
    t_start = time.time()
    model = gen_and_validate([cell,cell],data,scale=5,cell_types=[1,0,0,1,0,0,1,0,0,1,0,0,1])
    if not model:
        return np.zeros(512),0
    else:
        loss, correct, preds,acc_preds = full_model_run(model,
                                              data=data,
                                              epochs=512,
                                              lr=.01,
                                              momentum=.9,
                                              weight_decay=1e-4,
                                              lr_schedule=lr_schedule512,
                                              drop_path=True,
                                              log=True,
                                              track_progress=True,
                                              prefix="Macro",
                                              verbose=False)
    
        return correct,time.time()-t_start

In [ ]:
load=False
if load:
    results = pkl.load(open('micro_macro_results.pkl','rb'))
else:
    results = []

try:
    for i in range(50):
        print("\n===== CELL {} ===== ".format(i))
        cell = gen_cell(np.random.randint(3,8),connectivity=np.random.uniform(.3,.7),concat=.5)
        micro_scores,micro_time = micro_net(cell)
        macro_scores,macro_time = macro_net(cell)
        results.append({'micro_scores':micro_scores,'macro_scores':macro_scores,'micro_time':micro_time,'macro_time':macro_time})
        pkl.dump(results,open('micro_macro_results.pkl','wb'))
        
        notify_me("Test {} successful, moving on. Micro: {}, Macro: {}".format(i,max(micro_scores),max(macro_scores)))
except KeyboardInterrupt as e:
    raise e
except Exception as e:
    notify_me("Micro/Macro test errored. {}".format(str(e)))
    raise e


===== CELL 0 ===== 
== micro ==
Validating model...[SUCCESS]
Estimated Timelines:
	1 epoch   : 14.86 s
	64 epochs : 15 min, 51 s
	128 epochs: 31 min, 42 s
	512 epochs: 2 hrs, 6 mins, 48 s
Number of parameters: 8,178
Micro   8/8: 40.11%, Predicted: 4011 (±0.00)2))
== macro ==

===== CELL 1 ===== 
== micro ==
Validating model...[SUCCESS]
Estimated Timelines:
	1 epoch   : 11.08 s
	64 epochs : 11 min, 49 s
	128 epochs: 23 min, 38 s
	512 epochs: 1 hrs, 34 mins, 33 s
Number of parameters: 5,990
Micro   8/8: 47.03%, Predicted: 4703 (±0.00)2))
== macro ==
Validating model...[SUCCESS]
Estimated Timelines:
	1 epoch   : 46.23 s
	64 epochs : 49 min, 18 s
	128 epochs: 1 hrs, 38 mins, 37 s
	512 epochs: 6 hrs, 34 mins, 28 s
Number of parameters: 788,762


In [ ]:
notify_me("Micro/Macro test finished!")

In [ ]:
a = [{'a':[50,100,150],'b':[1000,1500,2000],'c':150,'d':5}]
pd.DataFrame.from_records(a)

In [ ]:
"{:,}".format(1.5e7)